In [1]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')
project_id = "gc1-rifqijulianhasyari" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


**NAMA : Rifqi Julian Hasyari**
**BATCH : HCK 006**

**Problem statement**
Apakah strategi marketing yang dilakukan sudah tepat untuk meningkatkan total penjualan dan juga total revenue dari perusahaan?

# **SOAL PENJABARAN**

**SOAL PENJABARAN 1**


> Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?



In [2]:
Complete = client.query('''
SELECT
        EXTRACT(month FROM created_at) AS Bulan,
        COUNT(status) AS Jumlah
FROM bigquery-public-data.thelook_ecommerce.order_items
WHERE
        EXTRACT(year FROM created_at) = 2022 AND
        status = 'Complete' AND EXTRACT(month FROM created_at) <= 9
GROUP BY Bulan
ORDER BY Bulan

''').to_dataframe()
Complete

,Bulan,Jumlah
0,1,917
1,2,902
2,3,1105
3,4,1053
4,5,1132
5,6,1334
6,7,1384
7,8,1386
8,9,1483




1.   Untuk menjawab pertanyaan di soal penjabaran 1, saya menggunakan tabel order items dikarenakan tabel tersebut memiliki data yang dibutuhkan. Saya menghitung setiap status complete yang muncul dengan menggunakan agregasi COUNT dan mengambil bulan yang ada pada kolom created_at dengan menggunakan EXTRACT. Terdapat 3 kondisi yang dibutuhkan yaitu dimana hanya status yang complete yang diambil, lalu data hanya diambil yang tahun 2022 serta hanya sampai bulan 9 atau Q3.Lalu saya melakukan GROUP by dan ORDER BY berdasarkan Bulan.
2.   Berdasarkan data tersebut dapat dilihat bahwa Data pembelian semakin meningkat di setiap kuartal nya. Hal ini menunjukan bahwa promosi yang dilakukan sudah tepat sasaran dan dapat lebih ditingkatkan untuk menghasilkan penjualan yang lebih meningkat juga.


**SOAL PENJABARAN 2**


> Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?



In [3]:
TotalPenjualan=client.query('''
  SELECT
        SUM(Revenue) as Total_Revenue,
        EXTRACT(MONTH FROM created_at) as Bulan
  from
  (
        SELECT *, num_of_item *  sale_price as Revenue
  FROM
  (     SELECT  items.created_at, orders.num_of_item, items.sale_price, orders.status
  FROM `bigquery-public-data.thelook_ecommerce.order_items` as items
  JOIN `bigquery-public-data.thelook_ecommerce.orders` as orders
  on items.order_id = orders.order_id) as subs1
  WHERE
        status = 'Complete'
        AND created_at BETWEEN '2022-01-01' AND '2022-09-30'
  ) as subs2
  GROUP BY Bulan
  order by Bulan

''').to_dataframe()
TotalPenjualan

,Total_Revenue,Bulan
0,112785.390055,1
1,108338.130215,2
2,124982.660281,3
3,116913.740171,4
4,122340.480044,5
5,161860.810129,6
6,157046.080015,7
7,155265.050136,8
8,159606.930057,9




1.   Untuk menjawab soal penjabaran 2, saya menggunakan 2 tabel yang saya lakukan join yaitu tabel order_items dan tabel orders. Lalu saya menggunakan 2 subquery. Untuk menghitung revenue saya melakukan perkalian num_of_item dengan sale_price namun yang diminta pada soal ialah total revenue. Maka dari itu setelah saya telah menghitung revenue saya menggunakan agregasi sum untuk menghitung total revenue yang ada di tiap bulan dengan kondisi status hanya yang complete dan diambil hanya sampai bulan 9 di tahun 2022.
2.   selisih kenaikan yang terjadi dari awal q1 hingga akhir q3 yaitu sebesar 46.821 USD. Hal ini berarti promosi yang dilakukan sudah tepat sasaran sehingga meningkatkan total revenue yang dimiliki perusahaan Namun peningkatan tersebut dirasa belum maksimal sehingga diharapkan untuk promosi yang dilakukan dapat ditingkatkan.



**SOAL PENJABARAN 3**


> Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?


In [4]:
Total_User = client.query('''
    SELECT
            COUNT(DISTINCT user_id) as total_user,
            EXTRACT(MONTH FROM created_at) as Bulan
    FROM (
      SELECT
            orders.user_id,
            orders.created_at,
            orders.status

      FROM `bigquery-public-data.thelook_ecommerce.users` as users
      JOIN `bigquery-public-data.thelook_ecommerce.orders` as orders
      ON users.id = orders.user_id
    ) as subs1
    WHERE
            status = 'Complete' AND
            created_at BETWEEN '2022-01-01' AND '2022-09-30'
    GROUP BY Bulan
    ORDER BY Bulan

''').to_dataframe()
Total_User

,total_user,Bulan
0,628,1
1,602,2
2,759,3
3,725,4
4,820,5
5,866,6
6,898,7
7,965,8
8,1014,9




1.   Untuk menjawab soal penjabaran 3 saya menggunakan tabel users dan orders. Kode yang dibuat hampir sama seperti di soal 2 namun disini saya menggunakan DISTINCT yang berfungsi untuk tidak mengambil user_id yang sama.
2.   Berdasarkan hasil perhitungan total user tersebut dapat disimpulkan bahwa total user yang melakukan transaksi mengalami peningkatan. Hal ini berarti promosi yang dilakukan sudah tepat dan menarik user baru untuk melakukan transaksi di perusahaan kita.



**Soal Penjabaran 4**


> Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!



In [5]:
Kategori_max = client.query('''

SELECT Bulan, category,total
FROM (
    SELECT
          Bulan,category,total,
        ROW_NUMBER() OVER (PARTITION BY Bulan ORDER BY total DESC) AS Urutan_partisi
    FROM (
        SELECT
          category,
          EXTRACT(MONTH FROM created_at) as Bulan,
          COUNT(products.id) as total

    FROM `bigquery-public-data.thelook_ecommerce.products` as products
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as items
    ON products.id = items.product_id
    WHERE
         status = 'Complete' AND
         created_at BETWEEN '2022-01-01' AND '2022-09-30'
    GROUP BY
        category,
        Bulan
    ) AS subs
) AS subs2
WHERE Urutan_partisi = 1
ORDER BY Bulan


''').to_dataframe()
Kategori_max

,Bulan,category,total
0,1,Tops & Tees,67
1,2,Tops & Tees,72
2,3,Intimates,80
3,4,Intimates,87
4,5,Intimates,82
5,6,Intimates,97
6,7,Fashion Hoodies & Sweatshirts,106
7,8,Intimates,121
8,9,Sleep & Lounge,108




1.   Untuk menjawab pertanyaan di soal penjabaran 3 hampir sama seperti di soal 2 namun yang berbeda disini saya menggunakan window function yaitu saya menggunakan OVER dan partition by. Hal ini dilakukan karena saya ingin melakukan partisi berdasarkan bulan lalu saya order berdasarkan kategori produk yang paling banyak di beli hingga paling sedikit dibeli di tiap bulannya
2.   Berdasarkan output di atas dapat dilihat kategori produk yang menjadi penjualan terbanyak di tiap bulan. Dimulai dari Tops & Tees di bulan januari hingga sleep&lounge di bulan ke 9. Dengan data tersebut kita dapat melakukan beberapa strategi untuk meningkatkan penjualan di Q9 seperti melakukan diskon pada kategori yang penjualannya rendah ataupun fokus kepada kategori yang memiliki penjualan tertinggi seperti melakukan colaborasi dengan brand lain untuk semakin meningkatkan penjualan pada kategori produk tersebut.


**SOAL PENJABARAN 5**


> Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!



In [6]:
TotalPenjualan=client.query('''

SELECT Bulan, category, Total_Revenue
FROM
(SELECT   Bulan, category,
          MAX(Revenue) AS Revenue, SUM(Revenue) AS Total_Revenue,
          ROW_NUMBER() OVER (PARTITION BY Bulan ORDER BY MAX(Revenue) DESC) AS Urutan_partisi
FROM
(
    SELECT  products.category, orders.num_of_item, items.order_id,
            EXTRACT(MONTH FROM items.created_at) AS Bulan,
            orders.num_of_item * items.sale_price AS Revenue

    FROM `bigquery-public-data.thelook_ecommerce.products` AS products
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS items
    ON products.id = items.product_id

    JOIN `bigquery-public-data.thelook_ecommerce.orders` AS orders
    ON items.order_id = orders.order_id

      WHERE items.status = 'Complete' AND
      items.created_at BETWEEN '2022-01-01' AND '2022-09-30'
) AS subs1

GROUP BY Bulan, category
ORDER BY Bulan, Urutan_partisi) as subs2
WHERE Urutan_partisi = 1
ORDER BY Bulan

''').to_dataframe()
TotalPenjualan

,Bulan,category,Total_Revenue
0,1,Suits & Sport Coats,9377.490019
1,2,Suits & Sport Coats,11851.579988
2,3,Outerwear & Coats,12458.340038
3,4,Outerwear & Coats,20379.010035
4,5,Blazers & Jackets,4081.929989
5,6,Active,9218.070006
6,7,Sleep & Lounge,8744.040007
7,8,Active,14855.840028
8,9,Leggings,3203.709992




1.   Pada soal ini konsepnya seperti menggabungkan dari beberapa soal sebelumnya. Konsep kode juga hampir sama namun disini saya menggunakan 2 join atau menggunakan 3 tabel yaitu tabel products,order_items,orders.
2.   Berdasarkan output yang didapatkan total_revenue terbesar disumbangkan dari kategori suits&sport coats pada bulan januari hingga kategori leggings di bulan 9. Untuk strategi yang dapat dicoba untuk lakukan untuk meningkatkan total revenue adalah melakukan colaborasi untuk produk outerwear & coats dikarenakan bulan selanjutnya pada Q4 adalah musim dingin. Sehingga kita dapat meningkatkan penujualan kategori produk tersebut dan meningkatkan total revenue di q4.
3. Lalu jika muncul pertanyaan kenapa hasil soal 4 dan 5 kategorinya berbeda. Hal ini bisa terjadi karena beberapa faktor salah satunya harga per barang yang berbeda antar kategori.



**SOAL PENJABARAN 6**
Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

In [7]:
import pandas as pd
result = pd.concat([Total_User,Complete, TotalPenjualan])
result

,total_user,Bulan,Jumlah,category,Total_Revenue
0,628,1,<NA>,NaN,NaN
1,602,2,<NA>,NaN,NaN
2,759,3,<NA>,NaN,NaN
3,725,4,<NA>,NaN,NaN
4,820,5,<NA>,NaN,NaN
5,866,6,<NA>,NaN,NaN
6,898,7,<NA>,NaN,NaN
7,965,8,<NA>,NaN,NaN
8,1014,9,<NA>,NaN,NaN
0,<NA>,1,917,NaN,NaN


Menurut pandangan saya yang paling berkaitan dengan jumlah user adalah jumlah transaksi. Karena jumlah transaksi berhubungan dengan invoice dan invoice berhubungan lgsg kepada user/customer.

**KESIMPULAN**


> Berdasarkan data yang sudah saya kumpulkan dari soal penjabaran saya dapat mengambil kesimpulan bahwa strategi marketing yang dilakukan sudah baik dan tepat sasaran. Hal ini dapat dilihat bahwa terdapat kenaikan dari keseluruhan dimulai dari total penjualan hingga total user yang melakukan transaksi. Namun, Kenaikan tersebut menurut pandangan saya terasa belum maksimal atau dapat lebih ditingkatkan untuk menyambut Q4 di tahun 2022. Terdapat beberapa strategi yang saya rekomendasikan seperti berfokus kepada kategori yang memiliki total penjualan tertinggi atau melakukan colaborasi terhadap influencer atau pun brand lain kepada kategori yang memiliki total penjualan rendah agar kategori tersebut dapat bersaing dengan kategori yang lainnya. Lalu saya juga memiliki rekomendasi strategi terhadap perusahaan untuk meningkatkan total penjualan dan total revenue di Q4, seperti yang kita ketahui bahwa Q4 merupakan akhir tahun dan pada akhir tahun merupakan musim dingin di beberapa negara khususnya di eropa. Dengan info tersebut kita dapat berfokus kepada kategori outerwear dan coats dengan melakukan colaborasi atau pun membuat inovasi baru pada kategori tersebut.



# **POINT PERTANYAAN**

**1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!**
Menurut saya problem statement yang saya buat sudah terjawab dengan data-data yang sudah saya dapatkan dan dapat terukur.

**2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?**



> Menurut pandangan saya berdasarkan analisis yang sudah saya lakukan saya rasa tidak masuk akal jika di awal kuartal 4 harus mencapai 250000(USD). Hal ini dapat dilihat dari total revenue terakhir yang ada pada bulan 9 yaitu sebesar $159606.Selisih dari 2 angka tersebut terpaut jauh jika kita melihat dari peningkatan rata-rata yang kita alami di tiap bulannya, mungkin saja target 250000(USD) dapat tercapai di akhir quartal 4 dengan catatan kita harus meningkatkan strategi marketing dan memaksimalkan strategi marketing yang saya rasa sudah tepat ini . Jika memang perusahaan memaksa dan menargetkan angka tersebut di awal q4 maka kita harus melakukan gebrakan besar atau langkah besar namun terdapat juga resiko di dalamnya. Salah satu yang bisa kita lakukan mungkin dengan melakukan colaborasi dengan superstar yang memiliki fanbase besar di seluruh dunia.



**3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!**



> Menurut data yang sudah dikumpulkan , seperti pada point kesimpulan bahwa keseluruhan data mengalami peningkatan seperti total revenue, lalu peningkatan yang menurut saya menjadi prospek yang baik adalah peningkatan jumlah user yang melakukan transaksi di perusahan kita meningkat. Dengan meningkatnya jumlah user menandakan bahwa perusahaan kita semakin dikenal dan tentu saja jika kita melakukan strategi-strategi yang kita siapkan untuk menghadapi q4 akan meningkatkan kembali total user dan juga akan meningkatkan total revenue yang dimiliki perusahaan. Maka dari itu perusahaan layak untuk mendapatkan investor baru.

